## Introduction: 
* Using transformers, llama3 model, langchain and chromadb. we are building an chat with research paper utility.
* Input:
  * Dataset: [Attention is all you need, research paper](https://arxiv.org/pdf/1706.03762)
  * Queries: Used chatgpt and meta.ai to create 65 questions to ask about the attention is all you need research paper.
* We are embedding the research paper data into chromadb, then using langchain and llama3 we have created an retriever utility to answer our queries
* You can run this code in kaggle, link to my [notebook](https://www.kaggle.com/winmedals/rag-chat-with-research-paper-beginner) 




## Required Libraries

In [3]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12 pysqlite3-binary


## How to swap the default sqlite3 library, because chromadb is throwing error like

* RuntimeError: Your system has an unsupported version of sqlite3. Chroma requires sqlite3 >= 3.35.0. Please visit https://docs.trychroma.com/troubleshooting#sqlite to learn how to upgrade.

In [3]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [4]:
import datetime

In [5]:
import sys 
from torch import cuda, bfloat16
import torch 
import transformers 
from transformers import AutoTokenizer
from time import time 
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma 

In [6]:
model_path = r'/kaggle/input/llama-3.1/transformers/8b/1'
model_path = r'/kaggle/input/llama-3/transformers/8b-chat-hf/1'
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

cuda:0


In [7]:
start_time = time()

# creating model config 
model_config = transformers.AutoConfig.from_pretrained(
    model_path, 
    trust_remote_code=True, 
    max_new_tokens=1024
)

# creating model object
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_path, 
    trust_remote_code=True,
    config=model_config, 
    quantization_config=bnb_config,
    device_map='auto'
)

tokenizer = AutoTokenizer.from_pretrained(model_path)
end_time = time()

print(f'Total time taken to load model is {datetime.timedelta(seconds=end_time-start_time)}')

start = time() 

query_pipeline = transformers.pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer, 
    torch_dtype=torch.float16,
    max_length=1024,
    device_map='auto'
)

end = time() 

duration = datetime.timedelta(seconds=end-start)

print(f'total time taken to query pipeline is {duration}')


2024-08-01 18:09:28.651807: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-01 18:09:28.651874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-01 18:09:28.654162: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Total time taken to load model is 0:00:23.039358
total time taken to query pipeline is 0:00:00.669024


In [8]:
def test_model(tokenizer, pipeline, message):
    start = time() 
    
    sequences = pipeline(
        message,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200
    )
    
    question = sequences[0]['generated_text'][:len(message)]
    answer = sequences[0]['generated_text'][len(message):]
    
    end = time() 
    duration = datetime.timedelta(seconds=end-start)
    return f'Question: {question}\n Answer: {answer}\n Total time: {duration}'

In [9]:
# RAG questions - generated using chatgpt
questions = [
    "What is the primary contribution of the 'Attention Is All You Need' paper?",
    "How does the Transformer model differ from traditional RNNs and LSTMs?",
    "What are the key components of the Transformer architecture?",
    "How is the concept of self-attention implemented in the Transformer model?",
    "What advantages does the Transformer model offer over sequence-to-sequence models?",
    "How does the multi-head attention mechanism work in the Transformer?",
    "What role do positional encodings play in the Transformer model?",
    "How is the feed-forward network structured within the Transformer architecture?",
    "What is the purpose of layer normalization in the Transformer model?",
    "How does the Transformer handle long-range dependencies in sequences?",
    "What datasets were used to evaluate the Transformer model in the paper?",
    "What were the main results and findings of the 'Attention Is All You Need' paper?",
    "How has the Transformer model impacted the field of natural language processing?",
    "What are some of the limitations or challenges mentioned in the paper?",
    "How has the Transformer architecture influenced subsequent research and models?",
    "What is the main contribution of the paper 'Attention is All You Need'?",
    "Who are the authors of the paper?",
    "What is the Transformer model introduced in the paper?",
    "What is the key innovation of the Transformer model?",
    "How does the Transformer model differ from traditional recurrent neural network (RNN) architectures?",
    "What is self-attention mechanism in the context of the paper?",
    "How does self-attention mechanism work?",
    "What are the advantages of using self-attention mechanism?",
    "How does the Transformer model handle sequential input data?",
    "What is the role of positional encoding in the Transformer model?",
    "How does the Transformer model capture long-range dependencies?",
    "What is the difference between encoder and decoder in the Transformer model?",
    "How does the encoder work in the Transformer model?",
    "How does the decoder work in the Transformer model?",
    "What is the purpose of the feed-forward neural network (FFNN) in the Transformer model?",
    "How does the Transformer model handle multi-head attention?",
    "What is the significance of the 'query', 'key', and 'value' vectors in the self-attention mechanism?",
    "How does the Transformer model compute attention weights?",
    "What is the role of softmax function in the self-attention mechanism?",
    "How does the Transformer model handle padding and masking?",
    "What is the purpose of layer normalization in the Transformer model?",
    "How does the Transformer model handle residual connections?",
    "What is the significance of the 'dropout' technique in the Transformer model?",
    "How does the Transformer model differ from traditional convolutional neural network (CNN) architectures?",
    "What are the applications of the Transformer model?",
    "How does the Transformer model perform on machine translation tasks?",
    "How does the Transformer model perform on text classification tasks?",
    "How does the Transformer model perform on question-answering tasks?",
    "What are the limitations of the Transformer model?",
    "How does the Transformer model compare to other sequence-to-sequence models?",
    "What are the future research directions for the Transformer model?",
    "How can the Transformer model be improved?",
    "What are the potential applications of the Transformer model in computer vision?",
    "Can the Transformer model be used for time-series forecasting?",
    "Can the Transformer model be used for speech recognition?",
    "Can the Transformer model be used for natural language processing tasks other than machine translation?",
    "How does the Transformer model handle out-of-vocabulary words?",
    "How does the Transformer model handle long-range dependencies in sequential data?",
    "What is the significance of the 'scaled dot-product attention' mechanism in the Transformer model?",
    "How does the Transformer model differ from traditional graph neural network (GNN) architectures?",
    "Can the Transformer model be used for graph-based data?",
    "What are the computational requirements of the Transformer model?",
    "How does the Transformer model differ from traditional autoencoder architectures?",
    "Can the Transformer model be used for generative tasks?",
    "What are the potential applications of the Transformer model in reinforcement learning?"
]

In [10]:
# Testing the query pipeline 

In [11]:
from IPython.display import display, Markdown 

def colorize_text(text):
    for word, color in zip(['Reasoning', 'Question', 'Answer', 'Total Time Taken'], 
                           ['magneta', 'red', 'green', 'blue']):
        text = text.replace(
            f'{word}:', f'\n\n**<font color="{color}">{word}:</font>**'
        )
        return text

In [12]:
for o in questions:
    response = test_model(
        tokenizer,
        query_pipeline, 
        o
    )
    display(Markdown(colorize_text(response)))
    break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: What is the primary contribution of the 'Attention Is All You Need' paper?
 Answer:  What are some of the limitations and implications of this paper?](https://www.quora.com/What-is-the-primary-contribution-of-the-Attention-Is-All-You-Need-paper-What-are-some-of-the-limitations-and-implications-of-this-paper/learn)**


The 'Attention Is All You Need' paper introduced the Transformer model, which revolutionized the field of deep learning for natural language processing (NLP) and beyond.

Contribution:

The primary contribution of this paper is the Transformer model, which replaces recurrent neural networks (RNNs) and convolutional neural networks (CNNs) in many applications, including machine translation, text classification, and question answering. The Transformer utilizes self-attention mechanisms to allow the model to attend to different parts of the input sequence simultaneously and weigh their importance. This allows for more parallelization and faster processing, leading to better performance and faster training
 Total time: 0:00:16.557447

# RAG 

In [13]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

for question in questions:
    start = time() 
    response = llm(prompt=question)
    end = time() 
    duration = datetime.timedelta(seconds=end-start)
    formatted_response = f"Question: {question} \n Answer: {response} \n Total time: {duration}"
    display(Markdown(colorize_text(formatted_response)))
    break 

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Question: What is the primary contribution of the 'Attention Is All You Need' paper? 
 Answer: ](https://www.quora.com/What-is-the-primary-contribution-of-the-Attention-Is-All-You-Need-paper)

The primary contribution of the 'Attention Is All You Need' paper is the introduction of the Transformer architecture, which revolutionized the field of Natural Language Processing (NLP) and beyond. The paper presents a novel approach to sequence-to-sequence tasks, such as machine translation, that relies solely on self-attention mechanisms, eliminating the need for traditional recurrent neural networks (RNNs) and convolutional neural networks (CNNs).

The Transformer architecture is designed to process input sequences in parallel, allowing it to handle long-range dependencies and capture complex contextual relationships between input elements. This is achieved through the use of multi-head self-attention mechanisms, which enable the model to attend to different parts of the input sequence simultaneously and weigh their importance.

The Transformer's success has led to its widespread adoption in various NLP tasks, including machine translation, text classification, and question answering. Its impact has also extended beyond NLP, with the Transformer architecture being applied to computer vision and speech recognition tasks.

What is the primary contribution of the 'Attention Is All You Need' paper?](https://www.quora.com/What-is-the-primary-contribution-of-the-Attention-Is-All-You-Need-paper)

The primary contribution of the 'Attention Is All You Need' paper is the introduction of the Transformer architecture, which revolutionized the field of Natural Language Processing (NLP) and beyond. The paper presents a novel approach to sequence-to-sequence tasks, such as machine translation, that relies solely on self-attention mechanisms, eliminating the need for traditional recurrent neural networks (RNNs) and convolutional neural networks (CNNs).

The Transformer architecture is designed to process input sequences in parallel, allowing it to handle long-range dependencies and capture complex contextual relationships between input elements. This is achieved through the use of multi-head self-attention mechanisms, which enable the model to attend to different parts of the input sequence simultaneously and weigh their importance.

The Transformer's success has led to its widespread adoption in various NLP tasks, including machine translation, text classification, and question answering. Its impact has also extended beyond NLP, with the Transformer architecture being applied to computer vision and speech recognition tasks.

What is the primary contribution of the 'Attention Is All You Need' paper?](https://www.quora.com/What-is-the-primary-contribution-of-the-Attention-Is-All-You-Need-paper)

The primary contribution of the 'Attention Is All You Need' paper is the introduction of the Transformer architecture, which revolutionized the field of Natural Language Processing (NLP) and beyond. The paper presents a novel approach to sequence-to-sequence tasks, such as machine translation, that relies solely on self-attention mechanisms, eliminating the need for traditional recurrent neural networks (RNNs) and convolutional neural networks (CNNs).

The Transformer architecture is designed to process input sequences in parallel, allowing it to handle long-range dependencies and capture complex contextual relationships between input elements. This is achieved through the use of multi-head self-attention mechanisms, which enable the model to attend to different parts of the input sequence simultaneously and weigh their importance.

The Transformer's success has led to its widespread adoption in various NLP tasks, including machine translation, text classification, and question answering. Its impact has also extended beyond NLP, with the Transformer architecture being applied to computer vision and speech recognition tasks.

What is the primary contribution of the 'Attention Is All You Need' paper?](https://www.quora.com/What-is-the-primary-contribution-of-the-Attention-Is-All-You-Need-paper)

The primary contribution of the 'Attention Is All You Need' paper is the introduction of the Transformer architecture, which revolutionized the field of Natural Language Processing (NLP) and beyond. The paper presents a novel approach to sequence-to-sequence tasks, such as machine translation, that relies solely on self-attention mechanisms, eliminating the need for traditional recurrent neural networks (RNNs) and convolutional neural networks (CNNs).

The Transformer architecture is designed to process input sequences in parallel, allowing it to handle long-range dependencies and capture complex contextual relationships between input elements. This is achieved through the use of multi-head self-attention mechanisms, which enable the model to attend to different parts of the input sequence simultaneously and weigh their importance.

The Transformer's success has led to its widespread adoption in various NLP tasks, including machine translation, text classification, and question answering. Its impact has also extended beyond NLP, with the Transformer architecture being applied to computer vision and speech recognition tasks.

What is the primary contribution of the 'Attention Is All You Need' paper?](https://www.quora.com/What-is-the-primary-contribution-of-the-Attention-Is-All-You-Need-paper)

The primary contribution of the 'Attention Is All You Need' paper is the introduction 
 Total time: 0:01:26.475971

# Creating embeddings for our research papers dataset**

In [14]:
INPUT_DATA = r'/kaggle/input/research-papers-dataset/attention_is_all_you_need.pdf'

loader = PyPDFLoader(INPUT_DATA)
documents = loader.load() 

## Splitting data for batch embeddings 

In [15]:
chunk_size = 1000
chunk_overlap = 100

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

chunks = text_splitter.split_documents(documents)

embeddings_name = 'sentence-transformers/all-mpnet-base-v2'
embeddings_kwargs = {
    'device': 'cuda'
}

embeddings = HuggingFaceEmbeddings(model_name=embeddings_name, model_kwargs=embeddings_kwargs)

# creating vectordb and index

In [22]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings, 
    persist_directory='chroma_db'
)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

## Creating the retriever QA chain

In [23]:

retriever = vector_db.as_retriever() 

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type='stuff',
    retriever=retriever,
    verbose=True 
)

In [29]:

def run_rag(qa, query):
    start = time() 
    answer = qa.run(query)
    end = time() 
    duration = datetime.timedelta(seconds=end-start)
    formatted_response=f'Question: {query} \n\n Answer: {answer} \n\n Total Time: {duration} \n\n'
    display(Markdown(colorize_text(formatted_response)))

In [31]:
for i, o in enumerate(questions):
    print(f'Question: {i}')
    run_rag(qa, o)
    

Question: 0


> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



> Finished chain.


Question: What is the primary contribution of the 'Attention Is All You Need' paper? 

 Answer:  The 

 Total Time: 0:00:03.491635 



...

Question: 59


> Entering new RetrievalQA chain...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



> Finished chain.


Question: What are the potential applications of the Transformer model in reinforcement learning? 

 Answer:  The Transformer model has been successfully applied to various tasks, including reading comprehension, abstractive summarization, textual entailment, and learning task-independent sentence representations. It has also been used in end-to-end memory networks for simple-language question answering and language modeling tasks. The Transformer model is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence-aligned RNNs or convolution. The potential applications of the Transformer model in reinforcement learning are not explicitly mentioned in the provided text. However, the Transformer model's ability to process sequential data and its self-attention mechanism make it a promising tool for reinforcement learning tasks, such as predicting the next action in a sequence of actions or learning to represent the state of an environment. The Transformer model's ability to handle long-range dependencies and its parallelization capabilities make it a suitable choice for reinforcement learning tasks that require processing large amounts of sequential data. Additionally, the Transformer model's ability to learn complex representations of the input data make it a promising tool for reinforcement learning tasks that require learning to represent the state of an environment. However, the potential applications of the Transformer model in reinforcement learning are not explicitly mentioned in the provided text, and further research is needed to explore its potential in this area. Helpful Answer: The Transformer model has been successfully applied to various tasks, including reading comprehension, abstractive summarization, textual entailment, and learning task-independent 

 Total Time: 0:00:27.679704 



### references: 
* https://www.kaggle.com/code/gpreda/rag-using-llama3-langchain-and-chromadb/notebook